# $\tau$-Disruptivity trajectories.
This document outlines some very rough work for plotting the $\tau$ disruptivity as a function of time. It is not recommended to look through this.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.optimize import minimize, LinearConstraint
from scipy.interpolate import interp1d, LinearNDInterpolator

# Move into the source directory for this notebook to work properly
# Probably want a better way of doing this.
import os
import importlib
os.chdir('../src/')

# Import whatever we need
import disruptivity as dis
import vis.disruptivity_vis as dis_vis
import vis.probability_vis as prob_vis
from vis.plot_helpers import plot_subplot as plot
import data_loader

# Import tokamak Configuartions
from tokamaks.cmod import CONFIG as CMOD
from tokamaks.d3d import CONFIG as D3D

importlib.reload(dis)
importlib.reload(dis_vis)
load_disruptions_mat = data_loader.load_disruptions_mat

Loading is the same as before, we use the premade functions for disruptivity computations.

In [ ]:
cmod_df, cmod_indices = load_disruptions_mat('../data/CMod_disruption_warning_db.mat')

In [ ]:
plt.plot(cmod_df.shot)
cmod_df.keys()

In [ ]:
'''
So my goal with this block of code is to find all the portions of flat top disrupted shots 
that are in flat tops. Should be simple enough.
'''

# Entry dictionary
entry_dict_1D = {
    'z_error': CMOD["entry_dict"]["z_error"],
    'kappa': CMOD["entry_dict"]["kappa"],
#     'z_error': CMOD["entry_dict"]["z_error"],
}

entry_dict_2D = {
    'q95':CMOD['entry_dict']['q95'],
    'n_e':CMOD['entry_dict']['n_e'],
}

# Hugill
# Compute the murakami parameter
cmod_df['inv_q95'] = 1/cmod_df['q95']
cmod_df['murakami'] = cmod_df['n_e']*0.68/(cmod_df['n_equal_1_mode']/cmod_df['n_equal_1_normalized'])/1e19

entry_dict_H = {
    'murakami':{
        'range':[0,20],
        'axis_name': "$n_e R/B_T \ (10^{19}$m$^{-2}$/T)",
    },
    'inv_q95':{
        'range':[0, 1],
        'axis_name': "$1/q_{95}$",
    },
 }

Now, we can reuse the histogram binning code for variable timesteps that returns the data indices of data points for each bin. Since this new method essentially tries to compute the dt of subsequent data points, it is mechanically the same as the dt calculation for variable timestep as well! 

In [ ]:

    
# Computing the disruptivity and plotting the regular figure
# shotlist=None
# entry_dict = entry_dict_2D
# indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(CMOD, cmod_df, cmod_indices, shotlist=shotlist)
# args = dis.compute_disruptivity_likelihood(cmod_df, entry_dict, indices_n_total, nbins=35, tau=50, window=25)
fig,ax = plot('cmod_q95_ne_disruptivity_kaloyannis.png', dis_vis.subplot_disruptivity2d, args, figsize=(7,4))

# Get a pulse's flat top data
shot = 1140226013 #AT PULSE
# shot = 1120105021 #VDE
# Plotting Constraints
# ax.plot([0,20], [0.5, 0.5], '--', c='orange')
# ax.plot([0,20], [0.0, 0.5], '--', c='orange')

In [ ]:
dis_vis.plot_data_selection(*args)

In [ ]:
def big_crunch(dataframe, indices, shotlist, tokamak, figtype, nbins=25, tau=0, window=2, dis_type='sampling'):
    
    # Figure settings
    figwidth = 6
    figheight = 4
    
    # Create the big plot
    n_plots = len(tokamak['entry_dict'])
    n_cols = 4
    # Make sure we don't make stupid plots that are all whitespace
    if n_plots < n_cols:
        n_cols = n_plots
    n_rows = np.ceil(n_plots/n_cols).astype(int)
    fig, ax = plt.subplots(n_rows, n_cols, figsize=(
        figwidth*n_cols, figheight*n_rows), constrained_layout=True)
    fig2, ax2 = plt.subplots(n_rows, n_cols, figsize=(
        figwidth*n_cols, figheight*n_rows), constrained_layout=True)
    
    # Compute the index histograms
    indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(tokamak, dataframe, indices, shotlist=shotlist, tau=tau, window=window)
    
    # Loop through the data fields
    # If we only have a single row
    for (i, entry) in enumerate(tokamak['entry_dict']):
        # Information Lines
        print("Working on "+entry)

        # Create the entry dict
        entry_dict = {entry:tokamak['entry_dict'][entry]}

        # Get the tokamak name
        name = tokamak['name']

        # Compute Disruptivity and save the plot
        if dis_type=='sampling':
            args = dis.compute_disruptivity_sampling(dataframe,
                                            entry_dict,
                                            indices_n_disrupt,
                                            indices_n_total,
                                            nbins=nbins)
        
        elif dis_type=='likelihood':
            args = dis.compute_disruptivity_likelihood(dataframe,
                                                       entry_dict,
                                                       indices_n_total,
                                                       nbins=nbins,
                                                       tau = tau,
                                                       window = window,
                                                      )
        else:
            assert False, f"Invalid calculation type {dis_type}"
            
        dis_vis.subplot_disruptivity1d(ax.flat[i], *args)

    # Remove axes of unrendered plots
    for i in range(n_plots, n_cols*n_rows):
        ax.flat[i].axis('off')
                
    return fig,ax


In [ ]:
# the big crunch
cmod_vde_shotlist = np.loadtxt("../data/cmod_vde_shotlist.txt", dtype=int)

# Parameter setup
# figtype = 'disruptivity_vde_kaloyannis'
figtype = 'disruptivity_likelihood'
shotlist = cmod_vde_shotlist #None # set to None for no shotlist

# Compute indices of interest
indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(CMOD, cmod_df, cmod_indices, shotlist=shotlist)

fig,ax = big_crunch(cmod_df, cmod_indices, shotlist, CMOD, figtype, nbins=25, tau=50, window=25)

In [ ]:
fig.savefig("big_crunch_vde.png", dpi=400, bbox_inches="tight")

### DIII-D Crunch

In [ ]:
import tokamaks.d3d as d3d
importlib.reload(d3d)
D3D = d3d.CONFIG
d3d_df, d3d_indices = load_disruptions_mat('../data/d3d-db-220420.mat')

In [ ]:
D3D

In [ ]:
# Hugill
# Compute the murakami parameter
d3d_df['inv_q95'] = 1/d3d_df['q95']
d3d_df['murakami'] = d3d_df['n_e']*0.68/(d3d_df['n_equal_1_mode']/d3d_df['n_equal_1_normalized'])/1e19

entry_dict_n1 = {
    "n1rms_normalized": D3D['entry_dict']['n1rms_normalized'],
    "n_equal_1_normalized": D3D['entry_dict']["n_equal_1_normalized"],
}

entry_dict_2D = {
    'q95':D3D['entry_dict']['q95'],
    'n_e':D3D['entry_dict']['n_e'],
}

In [ ]:
# Computing the disruptivity and plotting the regular figure
shotlist=None
entry_dict = entry_dict_2D
indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(D3D, d3d_df, d3d_indices, shotlist=shotlist)
args = dis.compute_disruptivity_likelihood(d3d_df, entry_dict, indices_n_total, nbins=35, tau=350, window=25)

In [ ]:
fig,ax = plot('d3d_n1_2d_disruptivity.png', dis_vis.subplot_disruptivity2d, args, figsize=(7,4))
# Get a pulse's flat top data
shot = 175552 #Cristina low n to locked mode disruption
ax.set_title(f"Shot {shot}")
dis_vis.subplot_draw_trajectory(ax, d3d_df, entry_dict, d3d_indices, shot)
# Plotting Constraints
# ax.plot([0,20], [0.5, 0.5], '--', c='orange')
# ax.plot([0,20], [0.0, 0.5], '--', c='orange')

In [ ]:
#Regular Trace
shot_data = d3d_df[d3d_df.shot==shot].index
# plt.plot(d3d_df['time'][shot_data],d3d_df['n_equal_1_normalized'][shot_data], label='n_equal_1_normalized')
# plt.plot(d3d_df['time'][shot_data],d3d_df['n1rms_normalized'][shot_data], label='n1rms_normalized')

plt.plot(d3d_df['time'][shot_data],d3d_df['q95'][shot_data], label='q95')
plt.plot(d3d_df['time'][shot_data],d3d_df['n_e'][shot_data]/1e19, label='n_e/1e19')

plt.legend()
plt.xlabel("Time (s)")

In [ ]:
# Parameter setup
figtype = 'disruptivity_likelihood'
shotlist = None # set to None for no shotlist
     
# Compute indices of interest
indices_n_disrupt, indices_n_total = dis.get_indices_disruptivity(D3D, d3d_df, d3d_indices, shotlist=shotlist)

# fig,ax = big_crunch(cmod_df, cmod_indices, shotlist, CMOD, figtype, nbins=25, tau=50, window=25)
fig,ax = big_crunch(d3d_df, d3d_indices, shotlist, D3D, figtype, nbins=35, tau=350, window=25)

In [ ]:
fig.savefig("big_crunch_d3d.png", dpi=400, bbox_inches="tight")

In [ ]:
plt.plot(d3d_df['z_error'])

In [ ]:
plt.plot(cmod_df['shot'][446000:475000])

In [ ]:
cmod_df['shot'][465000]

In [ ]:
# def subplot_trajectory()

In [ ]:
cmod_df.keys()

In [ ]:
test = np.array(cmod_df['n_e']/1e19)
test[test>70] = 0
test[test<0] = 0
plt.plot(test)

In [ ]:
plt.hist(cmod_df['n_e']/1e19, range=[0,50])

In [ ]:
cmod_vde_shotlist = np.loadtxt("../data/cmod_vde_shotlist.txt", dtype=int)

In [ ]:
cmod_vde_shotlist[0]

In [ ]:
d3d_df.keys()

In [ ]:
# plt.hist(d3d_df['n_equal_1_normalized'], bins=35)
plt.hist(d3d_df['n_equal_1_normalized'], bins=35, range=[0,1e-3])